In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import fastcluster

In [ ]:
ft = pd.read_csv("./HZV029_RP_neg_preferred.tsv", sep="\t")

In [ ]:
sns.kdeplot(np.array(np.log10(ft["peak_area"])), bw=0.5)
plt.xlabel("Log10 Peak Area", fontsize=18)
plt.ylabel("Density", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.show()


In [ ]:
sns.kdeplot(np.array(ft["goodness_fitting"]), bw_method=.1, common_norm=True)
plt.xlabel("Goodness Fitting", fontsize=18)
plt.ylabel("Density", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.show()

In [ ]:
sns.kdeplot(np.array(ft["cSelectivity"]), bw=0.1)
plt.xlabel("cSelectivity", fontsize=18)
plt.ylabel("Density", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.show()

In [ ]:
sns.kdeplot(np.array(np.log10(ft["snr"])), bw=0.1)
plt.xlabel("Log10 SNR", fontsize=18)
plt.ylabel("Density", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.show()

In [ ]:
sns.kdeplot(np.array(ft["detection_counts"]), bw=0.1)
plt.xlabel("Detection Counts", fontsize=18)
plt.ylabel("Density", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.show()

In [ ]:
import json
import matplotlib.colors as mcolors
from sys import setrecursionlimit
import scipy
setrecursionlimit(100000)

ft = pd.read_csv("./HZV029_RP_neg_preferred.tsv", sep="\t")
exp_obj = json.load(open("./HZV029_RP_neg_batch_corrected_experiment.json"))

color_map = {
    'blank': 'k',
    'blank_standard': 'grey',
    'blank_standard_dda': 'yellow',
    'qstd_standard': 'red',
    'qstd_standard_dda': 'orange',
    'nist': 'green',
    'pooled': 'purple',
    'pooled_dda': 'pink',
    'unknown': 'plum',
    'pooled': 'darkviolet'
}
batch_color_map = {
    1: "mistyrose",
    2: "lightcoral",
    3: "coral",
    4: "lightsalmon",
    5: "salmon",
    6: "darksalmon",
    7: "tomato",

    8: "lightcyan",
    9: "powderblue",
    10: "lightblue",
    11: "lightskyblue",
    12: "skyblue",
    13: "deepskyblue",
    14: "cornflowerblue",
    15: "dodgerblue",
    16: "royalblue",
    17: "blue"
}

col_colors = [[], []]
sample_columns = ft.columns[11:]
to_include = []
for sample in sample_columns:
    og_sample = sample
    sample = sample #.split("___")[-1]
    found = False
    for acquisition in exp_obj["acquisitions"]:
        if sample in acquisition['metadata_tags']['File Name']:
            if acquisition['metadata_tags']['Sample Type'] in ['pooled', 'unknown']:
                found = True
                col_colors[0].append(color_map[acquisition['metadata_tags']['Sample Type']])
                col_colors[1].append(batch_color_map[int(acquisition['metadata_tags']['batch'])])
                to_include.append(og_sample)
                break
assert(len(to_include) == len(col_colors[0]))
            
working_table = ft.copy()
corr_matrix = np.zeros((len(to_include), len(to_include)))
working_table = np.log2(working_table[to_include] + 1)
for i, s1 in enumerate(to_include):
    val_s1 = working_table[s1]
    for j, s2 in enumerate(to_include):
        if corr_matrix[j][i] != 0:
            corr_matrix[i][j] = corr_matrix[j][i]
        else:
            corr = np.corrcoef(val_s1, working_table[s2])
            corr_matrix[i][j] = corr[0][1]
    


In [ ]:
sns.clustermap(corr_matrix, col_colors=col_colors, cmap='coolwarm')

In [ ]:
df = []
for i, s1 in enumerate(to_include):
    row = {"sample": s1}
    for j, s2 in enumerate(to_include):
        row[s2] = corr_matrix[i][j]
    df.append(row)
pd.DataFrame(df).to_csv("correlation_matrix_wo_batch_corr.csv", index=False)

In [ ]:
import random

normalized = pd.read_csv("./HZV029_RP_neg_normalized.tsv", sep="\t")
batch_corrected = pd.read_csv("./HZV029_RP_neg_batch_corrected.tsv", sep="\t")
print(batch_corrected)

in_both = []
for x in normalized.columns:
    if x in ft.columns[11:] and x in normalized.columns and x in batch_corrected.columns:
        in_both.append(x)

sample_color_by_batch = {}
sample_color_by_type = {}
for acquisition in exp_obj["acquisitions"]:
    sample_color_by_batch[acquisition['metadata_tags']["File Name"].rstrip(".mzML")] = batch_color_map[int(acquisition['metadata_tags']['batch'])]


to_plot = random.sample(in_both, 100)
selected_ft = ft[to_plot]
TICs = np.nansum(selected_ft, axis=0)
log_TICs = np.log10(TICs)
plt.ylim([9.5, 11])
plt.bar(to_plot, log_TICs, color=[sample_color_by_batch[y] for y in to_plot])
plt.show()

selected_ft = normalized[[x for x in to_plot]]
TICs = np.nansum(selected_ft, axis=0)
log_TICs = np.log10(TICs)
plt.ylim([9.5, 11])
plt.bar(to_plot, log_TICs, color=[sample_color_by_batch[y] for y in to_plot])
plt.show()

selected_ft = batch_corrected[[x for x in to_plot]]
TICs = np.nansum(selected_ft, axis=0)
log_TICs = np.log10(TICs)
plt.ylim([9.5, 11])
plt.bar(to_plot, log_TICs, color=[sample_color_by_batch[y] for y in to_plot])
plt.show()



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

ft = pd.read_csv("./HZV029_RP_neg_preferred.tsv", sep="\t")
batch_lookup = {}

def do_PCA(table, skip_log=True):
    """
    Perform PCA on provided feature table, optionally log transform
    it first. 

    :param figure_params: dictionary with figure params

    :return: QAQC_result dict
    """
    samples = table.columns[11:]
    colors = []
    for s in samples:
        if s not in batch_lookup:
            batch_no = int(s.split("batch")[-1].split('_')[0])
            colors.append(batch_color_map[batch_no])
            batch_lookup[s] = batch_no
            batch_lookup[s.split('___')[-1]] = batch_no
        else:
            print(batch_lookup)
            colors.append(batch_lookup[s])

    sample_ftable = table[samples].T.copy()
    scaler = StandardScaler()
    pca_embedder = PCA(n_components=2)
    #if not skip_log:
    #    sample_ftable = np.log2(sample_ftable+1)
    pca_embedding = pca_embedder.fit_transform(scaler.fit_transform((sample_ftable)))
    Xs = [z[0] for z in pca_embedding]
    Ys = [z[1] for z in pca_embedding]
    print(colors)
    plt.scatter(Xs, Ys, c=colors)
    plt.show()

do_PCA(ft)

In [ ]:
import matplotlib.pyplot as plt

bc = pd.read_csv("./HZV029_RP_neg_batch_corrected.tsv", sep="\t")
plt.clf()
def do_PCA(table):
    """
    Perform PCA on provided feature table, optionally log transform
    it first. 

    :param figure_params: dictionary with figure params

    :return: QAQC_result dict
    """
    samples = table.columns[11:]
    colors = []
    for s in samples:
        colors.append(batch_color_map[batch_lookup[s]])

    sample_ftable = table[samples].T.copy()
    scaler = StandardScaler()
    pca_embedder = PCA(n_components=2)
    pca_embedding = pca_embedder.fit_transform(scaler.fit_transform((sample_ftable)))
    Xs = [z[0] for z in pca_embedding]
    Ys = [z[1] for z in pca_embedding]
    for x,y,c in zip(Xs, Ys, colors):
        print(x,y,c)

    plt.scatter(Xs, Ys, c=colors)
    plt.show()

do_PCA(bc)

In [ ]:
import pymzml
from intervaltree import IntervalTree
import matplotlib.pyplot as plt

def TIC(filepath_1, filepath_2):
    for path, color in zip([filepath_1, filepath_2], ['r', 'k']):
        mz_trees = [IntervalTree()] 
        rt_trees = [IntervalTree()] 
        mz_trees[0].addi(-np.inf, np.inf)
        rt_trees[0].addi(-np.inf, np.inf)
        bins = [[] for _ in mz_trees]
        rtimes = []
        for spec in pymzml.run.Reader(path):
            rtime = round(spec.scan_time[0] * 60, 3)
            rtimes.append(rtime)
            for bin in bins:
                bin.append(0)
            matches = [True if rt_tree.at(rtime) else False for rt_tree in rt_trees]
            match_mask = [i for i, match in enumerate(matches) if match]
            if match_mask:
                for peak in spec.peaks("centroided"):
                    mz = peak[0]
                    for match in match_mask:
                        if mz_trees[match].at(mz):
                            bins[match][-1] += float(peak[1])
        for i, bin in enumerate(bins):
            plt.plot(rtimes, bin, c=color)
    plt.ylabel("Intensity", fontsize=18)
    plt.xlabel("Retention Time (sec)", fontsize=18)
    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)
    plt.show()

TIC("./batch5_MT_20210730_001.mzML", "./batch5_MT_20210730_003.mzML")

In [ ]:
TIC("/Users/mitchjo/pcpfm/Analyses/Lipidomics_HR_LR/converted_acquisitions/MT_20230807_071.mzML", "/Users/mitchjo/pcpfm/Analyses/Lipidomics_HR_LR/converted_acquisitions/MT_20230807_069.mzML")

In [ ]:
hr_lr_exp = json.load(open("./HZV029_QC_experiment.json"))
results = hr_lr_exp["qcqa_results"]["masked_preferred_unknowns"]['feature_count_z_scores']["Result"]

X = []
Y = []
Cs = []
for i, (key, value) in enumerate(results.items()):
    X.append(i)
    Y.append(value)
    if key == "batch5_MT_20210730_001":
        Cs.append('r')
    elif key == "batch5_MT_20210730_003":
        Cs.append('blue')
    else:
        Cs.append('k')


plt.scatter(X, Y, c=Cs)

In [ ]:
hr_lr_exp = json.load(open("./HZV029_lipidomics_experiment.json"))
results = hr_lr_exp["qcqa_results"]["masked_preferred_unknowns"]['feature_count_z_scores']["Result"]

X = []
Y = []
Cs = []
for i, (key, value) in enumerate(results.items()):
    X.append(i)
    Y.append(value)
    if key == "MT_20230807_071":
        Cs.append('r')
    elif key == "MT_20230807_069":
        Cs.append('blue')
    else:
        Cs.append('k')


plt.scatter(X, Y, c=Cs)